In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')
%reload_ext watermark
%watermark -a "Izzet Turkalp Akbasli" -vmp langchain,openai

Author: Izzet Turkalp Akbasli

Python implementation: CPython
Python version       : 3.9.19
IPython version      : 8.12.0

langchain: 0.2.5
openai   : 1.34.0

Compiler    : GCC 12.3.0
OS          : Linux
Release     : 6.5.0-35-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 24
Architecture: 64bit



In [2]:
from unstructured.chunking.title import chunk_by_title
from unstructured.partition.md import partition_md
from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import dict_to_elements
import langchain, groq, fastembed, qdrant_client, unstructured
import unstructured.partition
%watermark --iversions

langchain    : 0.2.5
unstructured : 0.14.6
fastembed    : 0.3.1
groq         : 0.9.0
qdrant_client: 1.9.1



In [3]:
from unstructured.partition.pdf import partition_pdf

# Specify the path to your PDF file
filename = "/home/turkalp/Desktop/POKEMON_RAG/data/data_for_tutorials/RTI.pdf"
#path = "images"

# Extract images, tables, and chunk text
pdf_elements = partition_pdf(
                                filename=filename,
                                extract_images_in_pdf=False,
                                strategy = "hi_res",
                                hi_res_model_name="yolox",
                                infer_table_structure=True,
                                chunking_strategy="by_title",
                                max_characters=3000,
                                #new_after_n_chars=3800,
                                combine_text_under_n_chars=200,
                                #extract_image_block_output_dir=path,
)

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 14}

In [5]:
element_dict = [el.to_dict() for el in pdf_elements]

unique_types = set()

for item in element_dict:
    unique_types.add(item['type'])

print(unique_types)

{'CompositeElement'}


In [6]:
pdf_elements[0].to_dict()

{'type': 'CompositeElement',
 'element_id': 'aea361db53cf2c1dd8671346a5159ce1',
 'text': 'NCBI Bookshelf. A service of the National Library of Medicine, National Institutes of Health.\n\nStatPearls [Internet]. Treasure Island (FL): StatPearls Publishing; 2024 Jan-.\n\nPediatric Bronchiolitis\n\nAuthors\n\n3 Evelyn N. Erickson ; Rupal T. Bhakta ; Magda D. Mendez .',
 'metadata': {'filetype': 'application/pdf',
  'languages': ['eng'],
  'last_modified': '2024-06-18T00:18:15',
  'page_number': 1,
  'orig_elements': 'eJy9lW9v0zAQxr+KlVcgzZb/xE6yvdpgiCE2Vazvqqly7EtjLXWqxB0UxHfHTjcxoEIwaXuX5/ycfHc/Xbz4lkEHa/Bh6Wx2jLKSS2aopFhIKXBOlcA1tTlWTBibzqA22RHK1hC01UHHnG+Z6fvBOq8DjJPu9K7fhmULbtWGGOFCVDHnPvzZ2dDGKFNSxOimdz6kvMWCUUmisZKkuDlCD5IxRfKki3hQ/qkne9TZuBsDrFMXM/cFuuuNNpB9jwcWApjger80nR7H5Wbo62ijRPKCphoa10HYbWDKnV1mU7F+tdWrqaNFBn6V3UzRMSzXvXWNg2lenPIcU4VZOaf0mJXHTKbsTcxc+u26hiF1mooI8CXNIrt6c3aBzvr+dmyhawg6RSMMd84A6hsUWkBXOpWqO/TR1YMedil+CdYZ5+Ho5+mFH4ML2zjzZHgPugstSXc/NHKlhyGa72Cebo4l/M5aWCZorQosF

In [8]:
chunk_elements = chunk_by_title((pdf_elements),combine_text_under_n_chars=20,max_characters=3000)
len(chunk_elements)

5

In [10]:
elements = chunk_by_title(pdf_elements) # you can play around with the chunk_by_title arguments


In [11]:
import os
from langchain_core.documents import Document
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.vectorstores import Qdrant
documents = []
for element in elements:
    metadata = element.metadata.to_dict()
    del metadata["languages"]
    metadata["source"] = metadata["filename"]
    documents.append(Document(page_content=element.text, metadata=metadata))

In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
qdrant_url = os.getenv("QDRANT_URL")
qdrant_api_key = os.getenv("QDRANT_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

In [14]:
embeddings = FastEmbedEmbeddings()


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [15]:
# this will take some time, patience is the key :)
vectorstore = Qdrant.from_documents(documents=documents,
                                    embedding = embeddings,
                                    url = qdrant_url,
                                    collection_name="rti",
                                    api_key=qdrant_api_key)

In [16]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [42]:
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_core.runnables import RunnableSequence
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.vectorstores import Qdrant
from langchain.embeddings import FastEmbedEmbeddings
from langchain.chains import LLMChain

# Define the prompt template
template = """You are a medical AI assistant specialized in answering questions about bronchiolitis in children.
You will be provided with excerpts from a long document and a related question.
Respond in a conversational manner.
If you don't know the answer, simply say, "Hmm, I'm not sure."
Do not attempt to make up an answer.
Question: {input}
=========
{context}
=========
Answer in Markdown:"""
prompt = PromptTemplate(template=template, input_variables=["input", "context"])

# Initialize the language model
llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")

# Create the document chain
doc_chain = load_qa_with_sources_chain(llm, chain_type="map_reduce")

# Create embeddings using FastEmbedEmbeddings
embeddings = FastEmbedEmbeddings()

# Set up the Qdrant retriever
vectorstore = Qdrant.from_documents(documents=documents,
                                    embedding=embeddings,
                                    url=qdrant_url,
                                    collection_name="rti",
                                    api_key=qdrant_api_key)

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

# Create the history-aware retriever
history_aware_retriever = create_history_aware_retriever(
    llm=RunnableSequence(prompt | llm),
    retriever=retriever,
    prompt=prompt
)

# Create the LLM chain for generating answers
qa_chain = LLMChain(
    llm=llm,
    prompt=prompt
)

# Example question to test the chain
question = "What is the most common cause of bronchiolitis in children?"
context = "Bronchiolitis is inflammation of the bronchioles usually caused by an acute viral illness. It is the most common lower respiratory tract infection in children younger than 2 years of age."
result = qa_chain.invoke({"input": question, "context": context, "chat_history": []})
print(result)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

{'input': 'What is the most common cause of bronchiolitis in children?', 'context': 'Bronchiolitis is inflammation of the bronchioles usually caused by an acute viral illness. It is the most common lower respiratory tract infection in children younger than 2 years of age.', 'chat_history': [], 'text': '**Respiratory Syncytial Virus (RSV) is the most common cause of bronchiolitis in children.**'}


In [44]:
question = "What are the most common symptoms of bronchiolitis in children?"
context = "Bronchiolitis is inflammation of the bronchioles usually caused by an acute viral illness. It is the most common lower respiratory tract infection in children younger than 2 years of age."
result = qa_chain.invoke({"input": question, "context": context, "chat_history": []})
print(result)

{'input': 'What are the most common symptoms of bronchiolitis in children?', 'context': 'Bronchiolitis is inflammation of the bronchioles usually caused by an acute viral illness. It is the most common lower respiratory tract infection in children younger than 2 years of age.', 'chat_history': [], 'text': '**The most common symptoms of bronchiolitis in children are:**\n\n* **Cough**: a persistent and often loud cough, which can be dry or produce mucus\n* **Wheezing**: a high-pitched whistling sound when breathing out\n* **Fast breathing rate**: rapid breathing, often accompanied by rapid heartbeat\n* **Flaring**: outward movement of the ribs and the movement of the chest wall with each breath\n* **Retraction**: the sucking in of the skin between the ribs with each breath\n* **Fussiness**: irritability and restlessness due to discomfort and breathing difficulties\n\nThese symptoms can vary in severity and may worsen at night or with feeding.'}


In [45]:
question = "What are the recommended management and treatment strategies for children with bronchiolitis?"
context = "Bronchiolitis is inflammation of the bronchioles usually caused by an acute viral illness. It is the most common lower respiratory tract infection in children younger than 2 years of age."
result = qa_chain.invoke({"input": question, "context": context, "chat_history": []})
print(result)

{'input': 'What are the recommended management and treatment strategies for children with bronchiolitis?', 'context': 'Bronchiolitis is inflammation of the bronchioles usually caused by an acute viral illness. It is the most common lower respiratory tract infection in children younger than 2 years of age.', 'chat_history': [], 'text': "**Recommended Management and Treatment Strategies for Children with Bronchiolitis**\n\nAccording to the American Academy of Pediatrics (AAP) and the Centers for Disease Control and Prevention (CDC), the recommended management and treatment strategies for children with bronchiolitis are:\n\n* **Supportive care**: Provide plenty of fluids, such as breast milk or formula, to help keep the child hydrated. For older children, offer clear fluids like water or electrolyte-rich beverages.\n* **Oxygen therapy**: Provide supplemental oxygen if the child's oxygen saturation levels are low (less than 92%).\n* **Humidified air**: Use a cool-mist humidifier or a vapor

In [112]:
# hybrid search in action
filter_retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1, "filter": {"source": "gpt4all.pdf"}}
)

In [113]:
filter_chain = ConversationalRetrievalChain(
    retriever=filter_retriever,
    question_generator=question_generator_chain,
    combine_docs_chain=doc_chain,
)

In [115]:
filter_chain.invoke({
    "question": "How was GPT4All-Snoozy developed ?",
    "chat_history": [],
    "filter": filter,
})["answer"]

"I'm happy to help!\n\nFINAL ANSWER: The president did not mention Michael Jackson.\n\nSOURCES:\n\nFINAL ANSWER: This Agreement is governed by English law.\n\nSOURCES: 28-pl\n\nFINAL ANSWER: The president did not mention Michael Jackson.\n\nSOURCES:\n\nFINAL ANSWER: GPT4All-Snoozy was developed using roughly the same procedure as the previous GPT4All models, but with a few key modifications.\n\nSOURCES: gpt4all.pdf"